<a href="https://colab.research.google.com/github/Gonzalo-Messina/Programacion-Concurrente/blob/main/M6_threads_java.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo de Threads en Java

## Creacion de archivos

In [ ]:
%%writefile App.java
import java.util.ArrayList;
import java.util.List;
import java.util.Map;
import java.util.Random;

public class App 
{
  public static final int MIN_VALUE = -32;
  public static final int MAX_VALUE = 32;
  public static final int NUMBER_OF_MATRICES = 2;

  public static void main(String[] args) throws Exception
  {
    // Inicializamos matrices
    int matrixSize = parseInteger(args);
    List<int[][]> matrices = initializeMatrices(NUMBER_OF_MATRICES, matrixSize, MIN_VALUE, MAX_VALUE);
    int[][] matrixA = matrices.get(0);
    int[][] matrixB = matrices.get(1);

    // Creamos la caluladora
    MatrixCalculatorFactory calculatorFactory = new MatrixCalculatorFactory();
    MatrixCalculator sequentialMatrixCalculator = calculatorFactory.createMatrixCalculator(MatrixCalculatorFactory.SEQUENTIAL_CALCULATOR);
    MatrixCalculator parallelMatrixCalculator = calculatorFactory.createMatrixCalculator((MatrixCalculatorFactory.PARALLEL_CALCULATOR));
    
    // Calculamos resultados
    int[][] sequentialResult = sequentialMatrixCalculator.multiply(matrixA, matrixB);
    int[][] parallelResult = parallelMatrixCalculator.multiply(matrixA, matrixB);

    // Mostramos por pantalla los resultados o el error si hubiera.
    if (compare(sequentialResult, parallelResult))
    {
      List<Map.Entry<String, int[][]>> labeledMatrices = List.of(
        Map.entry("Matriz A", matrixA),
        Map.entry("Matriz B", matrixB),
        Map.entry("Resultado secuencial", sequentialResult),
        Map.entry("Resultado paralelo", parallelResult)
      );
      printMatrices(labeledMatrices);
    } 
    else
    {
      System.out.println("Las dos matrices no dieron igual!!");
    }
  }
  
  public static int parseInteger(String args[])
  {
    int squareMatrixSize = Integer.MIN_VALUE;
    try
    {
      squareMatrixSize = Integer.parseInt(args[0]);
      if(squareMatrixSize < 5 || squareMatrixSize > 20) 
      {
        System.err.println("El tamaño de la matriz debe ser un número entero entre 5 y 20");
        System.exit(1);
      }
    }
    catch (NumberFormatException e)
    {
      System.err.println("El primer parámetro debe ser un número entero");
      System.exit(1);
    }
    return squareMatrixSize;
  }
  
  // Inicializa una matrix cuadrada de tamaño n con números aleatorios entre min (inclusivo) y max (inclusivo)
  public static int[][] initializeMatrix(int n, int min, int max)
  {
    int[][] matrix = new int[n][n];
    Random randomizer = new Random();
    for(int i = 0; i < n; i++)
    {
      for(int j = 0; j < n; j++)
      {
        matrix[i][j] = randomizer.nextInt((max + 1) - min) + min;
      }
    }
    return matrix;
  }

  // Inicializa una lista de matrices cuadradas de tamaño n con números aleatorios entre min (inclusivo) y max (inclusivo)
  public static List<int[][]> initializeMatrices(int numberOfMatrices, int matrixDimension, int minValue, int maxValue)
  {
    List<int[][]> matrices = new ArrayList<>();
    for(int i = 0; i < numberOfMatrices; i++)
    {
      matrices.add(initializeMatrix(matrixDimension, minValue, maxValue));
    }
    return matrices;
  }
  

  // Compara dos matrices y devuelve true si son iguales, false en caso de que cualquier elemento de igual indice sea distinto
  public static boolean compare(int[][] matrixA, int[][] matrixB)
  {
    for(int i = 0; i < matrixA.length; i++)
    {
      for(int j = 0; j < matrixA.length; j++)
      {
        if(matrixA[i][j] != matrixB[i][j])
        {
          return false;
        }
      }
    }
    return true;
  }
  
  // Recibe una matriz y la imprime por consola
  public static void printMatrix(int[][] matrix)
  {
    for(int i = 0; i < matrix.length; i++)
    {
      System.out.print("|");
      for(int j = 0; j < matrix.length; j++)
      {
        System.out.print(String.format("%5d|",matrix[i][j]));
      }
      System.out.println();
    }
    System.out.println();
  }

    // Recibe una lista de tuplas de nombre, matriz y las imprime por consola
  public static void printMatrices(List<Map.Entry<String, int[][]>> matrices)
  {
    for(Map.Entry<String, int[][]> entryMap : matrices)
    {
      System.out.println(entryMap.getKey());
      printMatrix(entryMap.getValue());
    }
  }
}


Writing App.java


ConcurrentMatrixMultiplicator es una estrategia de multiplicacion que consiste en crear una pool de tantos Threads como filas tengan las matrices cuadradas y luego iniciarlos todos de manera que iteren por las columnas de sus filas para hacer la multiplicación con la otra matriz.

In [ ]:
%%writefile ConcurrentMatrixMultiplicator.java
import java.util.ArrayList;
import java.util.List;

public class ConcurrentMatrixMultiplicator implements IMatrixMultiplicationStrategy
  {
    public int[][] matrixMultiplication(int[][] a, int[][] b) {
      int rowsA = a.length;
      int rowsB = b.length;
      int[][] ch = new int[rowsA][rowsB];
      
      // LLevamos registro de todos los threads lanzados para esperarlos antes de devolver el resultado
      List<Thread> threadPool = new ArrayList<Thread>(a.length*b.length);
      for (int i = 0; i < a.length; i++) {
        // Las variables en el Thread tienen que ser inmutables.
        final int iCopy = i;
        Thread t = new Thread(() -> {
          for (int j = 0; j < b[0].length; j++) {
            for (int k = 0; k < a[0].length; k++) {
              ch[iCopy][j] += a[iCopy][k] * b[k][j];
  
            }
          }
        });

        // Agregamos el thread a la pool para lanzarlo luego.
        // Si bien podriamos lanzar el hilo recien creado ahora mismo, testeos locales con N > 1000 tuvieron mejor rendimiento
        //  al crear la pool con todos los hilos y lanzarlos luego.
        threadPool.add(t);
      }

      // Iniciamos los hilos en la pool.
      threadPool.forEach(t -> t.start());

      // Esperamos los hilos que todavía estén procesando la multiplicación
      threadPool.forEach(t -> {
        try 
        {
          t.join();
        } catch (InterruptedException e)
        {
          System.out.println("Ocurrio un error durante la multiplicacion en paralelo");
          e.printStackTrace();
        }
      });

      return ch;
    }
  }

Writing ConcurrentMatrixMultiplicator.java


In [ ]:
%%writefile IMatrixMultiplicationStrategy.java
public interface IMatrixMultiplicationStrategy
{
  // Returns product of two matrices
  public int[][] matrixMultiplication(int[][] a, int[][] b);
}

Writing IMatrixMultiplicationStrategy.java


Calculadora de matrices que utiliza una estrategia para hacer los calculos

In [ ]:
%%writefile MatrixCalculator.java
public class MatrixCalculator
{
  private IMatrixMultiplicationStrategy multiplicationStrategy_;

  public MatrixCalculator(IMatrixMultiplicationStrategy multiplicationStrategy) 
  {
    this.multiplicationStrategy_ = multiplicationStrategy;
  }

  public int[][] multiply(int[][] a, int[][] b)
  {
    return this.multiplicationStrategy_.matrixMultiplication(a, b);
  }
}

Writing MatrixCalculator.java


Factory de calculadora de matrices que setea las estrategias según se indique en el parametro

In [ ]:
%%writefile MatrixCalculatorFactory.java
public class MatrixCalculatorFactory
  {
    public static final String SEQUENTIAL_CALCULATOR = "sequential";
    public static final String PARALLEL_CALCULATOR = "parallel"; 

    public MatrixCalculator createMatrixCalculator(String strategy)
    {
      MatrixCalculator calculator = null;
      switch (strategy) {
        case SEQUENTIAL_CALCULATOR:
          calculator = new MatrixCalculator(new SequentialMatrixMultiplicator());
          break;
        case PARALLEL_CALCULATOR:
          calculator = new MatrixCalculator(new ConcurrentMatrixMultiplicator());
          break;
        default:
          System.out.println("Por favor, ingrese un algoritmo de multiplicacion valido");
          break;
      }
      return calculator;
    }
}

Writing MatrixCalculatorFactory.java


SequentialMatrixMultiplicator es una estrategia de multiplicación que consiste en iterar secuencialmente por todos los elementos de una matriz y hacer la multiplicación con la otra

In [ ]:
%%writefile SequentialMatrixMultiplicator.java
public class SequentialMatrixMultiplicator implements IMatrixMultiplicationStrategy
  {
    public int[][] matrixMultiplication(int[][] a, int[][] b) {
      //Multiply matrix a by matrix b
      int rowsA = a.length;
      int rowsB = b.length;
      int[][] cs = new int[rowsA][rowsB];
      
      for (int i = 0; i < a.length; i++) {
        for (int j = 0; j < b[0].length; j++) {
          for (int k = 0; k < a[0].length; k++) {
            cs[i][j] += a[i][k] * b[k][j];
          }
        }
      }
      return cs;
    }
  }

Writing SequentialMatrixMultiplicator.java


## Compilacion

In [ ]:
!javac *.java

## Ejecucion

In [ ]:
!java App 10

Matriz A
|   14|   21|    2|  -30|  -25|    1|   18|   16|   22|   32|
|  -23|   -4|   29|  -19|   12|   26|  -32|   11|   31|  -28|
|   15|  -19|  -11|    0|   -5|  -25|  -22|   -2|    2|  -25|
|  -14|   27|  -32|  -14|  -25|  -22|   -2|  -10|   12|  -25|
|   25|   13|  -10|  -13|  -18|   -2|   -7|    0|   23|   10|
|  -15|  -12|  -31|  -18|    2|   12|   20|  -31|    8|   28|
|   24|   28|   -3|   -3|   24|  -15|  -19|  -26|   10|   -6|
|   23|  -19|   28|   17|   -2|   16|    6|    4|  -27|   13|
|   26|  -31|   14|   29|    2|   13|    2|  -16|    5|  -28|
|  -11|  -23|   18|   -1|  -27|  -19|   25|   -2|    4|  -27|

Matriz B
|  -14|   18|    0|  -32|   18|   -3|   18|    8|    1|   25|
|  -23|  -23|    7|   20|  -19|    3|   16|    5|  -20|   17|
|  -14|  -28|   -6|    6|  -23|  -31|  -14|   -7|   11|   -3|
|   23|   27|  -29|   24|  -18|   -2|   25|    7|  -19|  -27|
|  -24|   -9|  -10|  -13|    2|   -9|   21|  -30|   29|   28|
|   12|   -7|    0|  -15|    3|  -31|    6|    1|  